In [1]:
import pandas as pd
import numpy as np
import pickle
from test_evaluate_functions.test_evaluate_functions import calculate_accuracies, test_model, variable_importance
from training_functions.training_functions import calculate_binary_variables, split, split_x_y, evaluate_models, train_ada_boost, split_easy_and_sudden_errors

In [2]:
dff = pd.read_csv("/home/leibniz/Desktop/IHMAN/preprocessing_results/out_eboxes.csv")
print("Reading done!")

Reading done!


# Drop nan columns:

In [3]:
# We will drop all the columns readings and lon and lat:
drop_cols = [
                col for col in dff.columns if 
                (col.startswith("power")) | (col.startswith("Active")) | (col.startswith("Reactive") | 
                (col in ["lat", "lon"])) | 
                (col == "Unnamed: 0") |
                (col.startswith("week")) |
                (col == "type") |
                (col == "ebox_id") |
                (col == "location") |
                (col == "id")
            ]
df = dff.drop(drop_cols, axis=1)
print("Drop done!")
# Interpolate with the mean in case it is necessary:
df = df.fillna(df.mean(numeric_only=True))
# Interpolation in this case should not be necessary

Drop done!


In [4]:
df = calculate_binary_variables(df)
# Preprocessing and split:
df["current_week"] = pd.to_datetime(df["current_week"])
train, validation, test = split(df, n_weeks=80)

cols_to_train = df.drop(["current_week", "hours_next_four_weeks", "error_next_four_weeks", "hours_week+1", "hours_week+2", "hours_week+3", "hours_week+4"], axis=1).columns
x_train, y_train = split_x_y(train, cols_to_train)
x_validation, y_validation = split_x_y(validation, cols_to_train)
x_test, y_test = split_x_y(test, cols_to_train)

0.6905388399496625
0.3274224917057545
0.3386340235670976
0.3339434847271479
2015-06-08 00:00:00
2020-01-20 00:00:00
2020-01-27 00:00:00
2021-08-02 00:00:00
2021-08-09 00:00:00
2023-02-20 00:00:00


Extract the datasets fro Avatar cognition:

In [8]:
train.to_csv("/home/leibniz/Desktop/IHMAN/avatar_cognition_datasets/eboxes_train.csv")
validation.to_csv("/home/leibniz/Desktop/IHMAN/avatar_cognition_datasets/eboxes_validation.csv")
test.to_csv("/home/leibniz/Desktop/IHMAN/avatar_cognition_datasets/eboxes_test.csv")

In [9]:
train

,hours_week-4,hours_week-3,hours_week-2,hours_week-1,current_week,hours_current_week,hours_week+1,hours_week+2,hours_week+3,hours_week+4,...,Wind_avg_std_current_week,Wind_min_min_current_week,Pres_max_max_current_week,Pres_avg_avg_current_week,Pres_avg_std_current_week,Pres_min_min_current_week,Precipitation_avg_current_week,Precipitation_sum_current_week,hours_next_four_weeks,error_next_four_weeks
6339,0.262222,0.000000,0.000000,0.000000,2019-10-14,0.000000,0.071111,0.000000,0.000000,0.000000,...,2.198376,0,30.1,29.885714,0.106904,29.7,0.0,0.0,0.071111,Yes
8083,0.000000,0.000000,0.000000,0.000000,2016-08-15,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.795224,0,28.2,28.000000,0.115470,27.9,0.0,0.0,0.000000,No
2984,0.000000,0.000000,0.000000,0.000000,2019-12-02,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.364691,0,30.3,29.900000,0.365148,0.0,0.0,0.0,0.000000,No
6967,0.000000,0.000000,0.034167,0.000000,2019-09-09,0.000000,0.010278,0.000000,0.000000,0.000000,...,2.508462,0,30.4,30.085714,0.195180,29.7,0.0,0.0,0.010278,Yes
1493,0.031389,0.000000,0.116667,0.088889,2019-07-08,0.110000,0.000000,0.000000,0.016389,0.030833,...,3.473437,0,30.0,29.900000,0.000000,29.8,0.0,0.0,0.047222,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6551,0.993889,0.000000,0.000000,0.000000,2019-10-21,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.897043,0,30.2,29.957143,0.161835,29.6,0.0,0.0,0.000000,No
4413,0.000000,0.076944,0.147500,0.113333,2019-02-18,0.000000,0.000000,0.031667,0.016111,0.058333,...,1.270171,0,30.5,30.257143,0.127242,30.0,0.0,0.0,0.106111,Yes
4448,0.010556,0.000000,0.000000,0.026944,2019-10-21,0.003333,0.093056,0.000000,0.005000,0.187778,...,2.210365,0,30.2,30.014286,0.157359,29.8,0.0,0.0,0.285833,Yes
5086,0.000000,0.000000,0.000000,0.000000,2019-12-16,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.154619,0,30.3,29.914286,0.195180,29.5,0.0,0.0,0.000000,No


### Train model with the whole train dataset:

In [39]:
model_results = evaluate_models(
    max_depth_l = [1],
    n_estimators_l = [10, 20],
    lr_l = [0.5, 0.7, 0.8],
    prob_threshold_l = np.arange(0.46, 0.5, 0.01).tolist(),
    x_train = x_train,
    y_train = y_train,
    x_validation = x_validation,
    y_validation = y_validation
)

/home/leibniz/anaconda3/envs/predic_maint/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/leibniz/Desktop/IHMAN/predictive_maintenance/training_functions/training_functions.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_results = model_results.append(row_to_append, ignore_index=True)
/home/leibniz/anaconda3/envs/predic_maint/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/leibniz/Desktop/IHMAN/predictive_maintenance/training_functions/training_functions.py:162: FutureWarnin

In [40]:
model_results.loc[(model_results["yes_accuracy"] >= model_results["yes_accuracy"].max()-0.37) & ((model_results["no_accuracy"] >= model_results["no_accuracy"].max()-0.37))]

,learning_rate,n_estimators,total_accuracy,yes_accuracy,no_accuracy,prob_threshold,tree_depth
0,0.5,10.0,0.641554,0.656104,0.633280,0.46,1.0
1,0.5,10.0,0.641554,0.656104,0.633280,0.47,1.0
2,0.5,10.0,0.653378,0.626281,0.668786,0.48,1.0
4,0.7,10.0,0.623649,0.688723,0.586645,0.46,1.0
5,0.7,10.0,0.623649,0.688723,0.586645,0.47,1.0
6,0.7,10.0,0.650676,0.631873,0.661367,0.48,1.0
9,0.8,10.0,0.624324,0.678472,0.593535,0.47,1.0
10,0.8,10.0,0.641892,0.630941,0.648119,0.48,1.0
14,0.5,20.0,0.563176,0.739049,0.463169,0.48,1.0
15,0.5,20.0,0.637500,0.627213,0.643349,0.49,1.0


We are going with the model lr=0.5, n_estimators=10, prob_threshold=0.48, tree_depth=1

In [61]:
ada_model = train_ada_boost(
    max_depth_tree = 1,
    n_estimators = 10,
    lr = 0.5,
    x  = x_train,
    y = y_train
)
# Test:
test_model(
    model = ada_model,
    x = x_test, 
    y = y_test,
    prob_threshold = 0.48
)

/home/leibniz/anaconda3/envs/predic_maint/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'total_accuracy': 0.7115450496745461,
 'yes_accuracy': 0.5689381933438986,
 'no_accuracy': 0.7508741258741258}

In [58]:
# Check of the importance of the variables:
feature_importances = variable_importance(
    ada_model = ada_model, 
    trained_columns = cols_to_train
)
feature_importances.loc[feature_importances["importance"] != 0]

/home/leibniz/Desktop/IHMAN/predictive_maintenance/test_evaluate_functions/test_evaluate_functions.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  variable_importance_df = variable_importance_df.append(row_to_append, ignore_index=True)
/home/leibniz/Desktop/IHMAN/predictive_maintenance/test_evaluate_functions/test_evaluate_functions.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  variable_importance_df = variable_importance_df.append(row_to_append, ignore_index=True)
/home/leibniz/Desktop/IHMAN/predictive_maintenance/test_evaluate_functions/test_evaluate_functions.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  variable_importance_df = variable_importance_df.append(row_to_append, ignore_index=True)
/home/leib

,variable,importance
0,hours_week-4,0.1
1,hours_week-3,0.1
2,hours_week-2,0.1
3,hours_week-1,0.1
4,hours_current_week,0.1
24,Pres_min_min_week-4,0.1
39,Wind_max_max_week-3,0.1
40,Wind_avg_avg_week-3,0.1
97,Dew_max_max_current_week,0.1
112,Pres_min_min_current_week,0.1


# Let's test with droping the nan rows and keeping the readings

In [45]:
# drop nan rows:
df = dff.loc[~dff["ActivePeak_current_week"].isna()]

# Drop some usless columns for the model:
drop_cols = [
    col for col in df.columns if
        (col in ["lat", "lon"]) | 
        (col == "Unnamed: 0") |
        (col.startswith("week")) |
        (col == "type") |
        (col == "ebox_id") |
        (col == "location") |
        (col == "id")
]
df = df.drop(drop_cols, axis=1)

# Interpolate some left missing values:
df = df.fillna(df.mean(numeric_only=True))

In [46]:
df = calculate_binary_variables(df)
# Preprocessing and split:
df["current_week"] = pd.to_datetime(df["current_week"])
train, validation, test = split(df, n_weeks=77)

cols_to_train = df.drop(["current_week", "hours_next_four_weeks", "error_next_four_weeks", "hours_week+1", "hours_week+2", "hours_week+3", "hours_week+4"], axis=1).columns
x_train, y_train = split_x_y(train, cols_to_train)
x_validation, y_validation = split_x_y(validation, cols_to_train)
x_test, y_test = split_x_y(test, cols_to_train)

0.6592943654555029
0.2456556082148499
0.37862032648762506
0.375724065297525
2015-06-08 00:00:00
2019-10-14 00:00:00
2019-10-21 00:00:00
2021-04-05 00:00:00
2021-04-12 00:00:00
2022-10-03 00:00:00


### Train model with the whole train dataset:

In [47]:
model_results = evaluate_models(
    max_depth_l = [1, 2, 3],
    n_estimators_l = [1, 2, 5, 10, 15],
    lr_l = [0.1, 0.2, 0.5],
    prob_threshold_l = np.arange(0.40, 0.55, 0.02).tolist(),
    x_train = x_train,
    y_train = y_train,
    x_validation = x_validation,
    y_validation = y_validation
)

/home/leibniz/anaconda3/envs/predic_maint/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/leibniz/Desktop/IHMAN/predictive_maintenance/training_functions/training_functions.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_results = model_results.append(row_to_append, ignore_index=True)
/home/leibniz/anaconda3/envs/predic_maint/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/leibniz/Desktop/IHMAN/predictive_maintenance/training_functions/training_functions.py:162: FutureWarnin

In [50]:
model_results.loc[(model_results["yes_accuracy"] >= model_results["yes_accuracy"].max()-0.33) & ((model_results["no_accuracy"] >= model_results["no_accuracy"].max()-0.33))]

,learning_rate,n_estimators,total_accuracy,yes_accuracy,no_accuracy,prob_threshold,tree_depth
42,0.2,5.0,0.661683,0.544666,0.739988,0.45,1.0
48,0.5,5.0,0.678025,0.572420,0.748694,0.45,1.0
49,0.5,5.0,0.678025,0.571552,0.749275,0.47,1.0
50,0.5,5.0,0.680111,0.497832,0.802089,0.49,1.0
60,0.2,10.0,0.664117,0.541197,0.746373,0.45,1.0
66,0.5,10.0,0.643255,0.609714,0.665699,0.45,1.0
67,0.5,10.0,0.644993,0.575022,0.691817,0.47,1.0
72,0.1,15.0,0.657858,0.587164,0.705165,0.45,1.0
78,0.2,15.0,0.641516,0.587164,0.677887,0.45,1.0
79,0.2,15.0,0.654033,0.542064,0.728961,0.47,1.0


We get slightly worse results if we go with the model lr=0.5, n_estimators=15, prob_threshold=0.47, tree_depth=1

In [52]:
ada_model_readings = train_ada_boost(
    max_depth_tree = 1,
    n_estimators = 15,
    lr = 0.5,
    x  = x_train,
    y = y_train
)
# Test:
test_model(
    model = ada_model_readings,
    x = x_test,
    y = y_test,
    prob_threshold = 0.47
)

/home/leibniz/anaconda3/envs/predic_maint/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'total_accuracy': 0.6741415557112824,
 'yes_accuracy': 0.5604395604395604,
 'no_accuracy': 0.7130761994355598}

We get slightly worse results so for the sake of simplicity we are going to just use the model that does not use the readings

# Export the model:

In [60]:
with open("predictive_models/ada_model_eboxes.pk1", "wb") as file:
    pickle.dump(ada_model, file)

# Save the probability thresholds:
with open("predictive_models/ada_prob_eboxes.pk1", "wb") as file:
    pickle.dump({"prob_ada_model": 0.48}, file)